In [398]:
import pandas as pd
import openpyxl
import numpy as np

In [399]:
data_14 = pd.read_csv('immo/2014.csv').drop(columns='Unnamed: 0')
data_15 = pd.read_csv('immo/2015.csv').drop(columns='Unnamed: 0')
data_16 = pd.read_csv('immo/2016.csv').drop(columns='Unnamed: 0')
data_17 = pd.read_csv('immo/2017.csv').drop(columns='Unnamed: 0')
data_18 = pd.read_csv('immo/2018.csv').drop(columns='Unnamed: 0')
data_19 = pd.read_csv('immo/2019.csv').drop(columns='Unnamed: 0')
data_20 = pd.read_csv('immo/2020.csv').drop(columns='Unnamed: 0')
data_21 = pd.read_csv('immo/2021.csv').drop(columns='Unnamed: 0')


In [400]:
data = pd.concat([data_14, data_15, data_16, data_17, data_18, data_19, data_20, data_21])
print(data.shape)
data = data.rename(columns={"INSEE_COM" : "Code INSEE"})
data.head()

(232354, 8)


,Code INSEE,Annee,Nb_mutations,NbMaisons,NbApparts,PrixMoyen,Prixm2Moyen,SurfaceMoy
0,01001,2014,10,10.0,0.0,180207.000000,1708.900,107.400000
1,01002,2014,3,3.0,0.0,107930.000000,1338.000,80.333333
2,01004,2014,125,77.0,48.0,164579.392000,1946.432,85.872000
3,01005,2014,15,12.0,3.0,252109.733333,2412.200,110.533333
4,01006,2014,1,1.0,0.0,60000.000000,857.000,70.000000


In [401]:
# code geo

code_geo = pd.read_csv('correspondance-code-insee-code-postal.csv', sep=";", usecols=["Code INSEE", "Commune", "geo_point_2d"])
code_geo.head()

,Code INSEE,Commune,geo_point_2d
0,59416,MORBECQUE,"50.67689342861573, 2.536216144331492"
1,22102,LANGOURLA,"48.284641107667674, -2.415501011324659"
2,31225,GOUTEVERNISSE,"43.214026301449536, 1.173765920286677"
3,23025,BONNAT,"46.32301933418213, 1.913450631364578"
4,38522,VALJOUFFREY,"44.88153491127053, 6.07950399615059"


In [402]:
data = data.merge(code_geo, how='left', on='Code INSEE')
data.head()

,Code INSEE,Annee,Nb_mutations,NbMaisons,NbApparts,PrixMoyen,Prixm2Moyen,SurfaceMoy,Commune,geo_point_2d
0,01001,2014,10,10.0,0.0,180207.000000,1708.900,107.400000,L'ABERGEMENT-CLEMENCIAT,"46.153425521369456, 4.926113542231843"
1,01002,2014,3,3.0,0.0,107930.000000,1338.000,80.333333,L'ABERGEMENT-DE-VAREY,"46.009187877601306, 5.428016963625987"
2,01004,2014,125,77.0,48.0,164579.392000,1946.432,85.872000,AMBERIEU-EN-BUGEY,"45.96084751144621, 5.372925777699918"
3,01005,2014,15,12.0,3.0,252109.733333,2412.200,110.533333,AMBERIEUX-EN-DOMBES,"45.99617998724963, 4.912272507964576"
4,01006,2014,1,1.0,0.0,60000.000000,857.000,70.000000,AMBLEON,"45.749498904386975, 5.594320173657713"


In [403]:
data.loc[data["Code INSEE"] == "75056", "Commune"] = "PARIS"
data.loc[data["Code INSEE"] == "13055", "Commune"] = "MARSEILLE"

In [404]:
# Detect the encoding of the file
import chardet

file_path = 'gares-tgv.csv'
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())
encoding = result['encoding']
encoding

'ISO-8859-1'

In [405]:
# TGV
tgv = pd.read_csv('gares-tgv.csv', encoding=encoding).drop(columns='Unnamed: 0')
tgv_list = tgv['Nom_Gare'].unique()
print(len(tgv_list))
tgv.head()

183


,Nom_Gare,INSEE_REG,NOM_REG,INSEE_DEP,NOM_DEP,Latitude,Longitude
0,Aix-en-Provence TGV,84,Auvergne-Rhône-Alpes,1,Ain,46.097,4.847
1,Bellegarde,84,Auvergne-Rhône-Alpes,1,Ain,46.114,5.824
2,Bourg-en-Bresse,84,Auvergne-Rhône-Alpes,1,Ain,46.196,5.275
3,Cluses,84,Auvergne-Rhône-Alpes,1,Ain,46.171,5.577
4,Nurieux,84,Auvergne-Rhône-Alpes,1,Ain,46.181,5.521


In [406]:
gares = pd.read_csv("liste-des-gares.csv", sep = ";", usecols=["libelle", "code_ligne", "commune", "voyageurs"]).rename(columns={"libelle" : "Nom_gare",
                                                                                                                    "commune" : "Commune"})
gares = gares[gares["voyageurs"] == "O"].reset_index(drop=True)
print(gares.shape)
gares.head()

(3303, 4)


,Nom_gare,voyageurs,code_ligne,Commune
0,Bandol,O,930000,BANDOL
1,Pouilly-sur-Loire,O,750000,POUILLY-SUR-LOIRE
2,Arbois,O,880000,ARBOIS
3,Nantes,O,519000,NANTES
4,Culmont-Chalindrey,O,843000,CHALINDREY


In [407]:
gares.tail()

,Nom_gare,voyageurs,code_ligne,Commune
3298,Labenne,O,655000,LABENNE
3299,Les Salelles,O,723000,LES SALELLES
3300,L'Isle-sur-le-Doubs,O,852000,L'ISLE-SUR-LE-DOUBS
3301,Balbigny,O,750000,BALBIGNY
3302,Montgeron-Crosne,O,830000,MONTGERON


In [408]:
for index, el in enumerate(gares["Nom_gare"]):
    if el in tgv_list:
        gares.loc[index, "TGV"] = "Gare TGV"
    else:
        gares.loc[index, "TGV"] = "Gare voyageurs non TGV"

In [409]:
gares[gares["TGV"] == "Gare voyageurs non TGV"].tail()

,Nom_gare,voyageurs,code_ligne,Commune,TGV
3298,Labenne,O,655000,LABENNE,Gare voyageurs non TGV
3299,Les Salelles,O,723000,LES SALELLES,Gare voyageurs non TGV
3300,L'Isle-sur-le-Doubs,O,852000,L'ISLE-SUR-LE-DOUBS,Gare voyageurs non TGV
3301,Balbigny,O,750000,BALBIGNY,Gare voyageurs non TGV
3302,Montgeron-Crosne,O,830000,MONTGERON,Gare voyageurs non TGV


In [410]:
gares["TGV"].unique()

array(['Gare voyageurs non TGV', 'Gare TGV'], dtype=object)

In [411]:
# merge avec data immo

data = data.merge(gares, how='left', on="Commune")
data['TGV'] = data['TGV'].fillna("Pas de gare")
data

,Code INSEE,Annee,Nb_mutations,NbMaisons,NbApparts,PrixMoyen,Prixm2Moyen,SurfaceMoy,Commune,geo_point_2d,Nom_gare,voyageurs,code_ligne,TGV
0,01001,2014,10,10.0,0.0,180207.000,1708.900,107.400000,L'ABERGEMENT-CLEMENCIAT,"46.153425521369456, 4.926113542231843",NaN,NaN,NaN,Pas de gare
1,01002,2014,3,3.0,0.0,107930.000,1338.000,80.333333,L'ABERGEMENT-DE-VAREY,"46.009187877601306, 5.428016963625987",NaN,NaN,NaN,Pas de gare
2,01004,2014,125,77.0,48.0,164579.392,1946.432,85.872000,AMBERIEU-EN-BUGEY,"45.96084751144621, 5.372925777699918",Ambérieu,O,889000.0,Gare voyageurs non TGV
3,01004,2014,125,77.0,48.0,164579.392,1946.432,85.872000,AMBERIEU-EN-BUGEY,"45.96084751144621, 5.372925777699918",Ambérieu,O,890000.0,Gare voyageurs non TGV
4,01004,2014,125,77.0,48.0,164579.392,1946.432,85.872000,AMBERIEU-EN-BUGEY,"45.96084751144621, 5.372925777699918",Ambérieu,O,883000.0,Gare voyageurs non TGV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237461,95678,2021,17,14.0,3.0,275319.400,3911.765,77.470590,VILLIERS-ADAM,"49.07028882476772, 2.239508718709138",NaN,NaN,NaN,Pas de gare
237462,95680,2021,186,67.0,119.0,174900.700,2575.919,69.456990,VILLIERS-LE-BEL,"49.00844941568135, 2.403856040394518",NaN,NaN,NaN,Pas de gare
237463,95682,2021,3,3.0,0.0,256666.700,2606.000,98.333330,VILLIERS-LE-SEC,"49.07430886817081, 2.386889599838927",NaN,NaN,NaN,Pas de gare
237464,95690,2021,2,2.0,0.0,338200.000,3532.000,98.500000,WY-DIT-JOLI-VILLAGE,"49.09581977581952, 1.8250288330776692",NaN,NaN,NaN,Pas de gare


In [412]:
ville = [
    "75056",  # Paris
    "69123",  # Lyon
    "13055",  # Marseille
    "33063",  # Bordeaux
    "59350",  # Lille
    "31555",  # Toulouse
    "44109",  # Nantes
    "67482",  # Strasbourg
    "35238",  # Rennes
    "06088",  # Nice
    "34172",  # Montpellier
    "38185",  # Grenoble
    "76351",  # Le Havre
    "51454",  # Reims
    "76540"   # Rouen
]



data[data["Code INSEE"].isin(ville)]
for index, el in enumerate(data["Code INSEE"]):
    if el in ville:
        data.loc[index, "TGV"] = "Gare TGV"


In [413]:
data.to_csv("TGV.csv", sep = ";", index=False)

In [414]:
import folium

paris = folium.Map(location = [46.8534, 2.3488], zoom_start=6,tiles = "https://{s}.tile.openstreetmap.fr/osmfr/{z}/{x}/{y}.png",
                  attr = '&copy; OpenStreetMap France | &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors')

In [415]:
print(df.shape)

(30876, 6)


In [416]:
# Augmenter le geojson

import json

with open('communes.geojson', 'r') as file:
    geojson_data = json.load(file)

for feature in geojson_data['features']:
    code = feature['properties']['code']
    # Ajouter la propriété "TGV" en fonction de la valeur correspondante dans votre DataFrame
    if code in data['Code INSEE'].values:
        tgv_value = data.loc[data['Code INSEE'] == code, 'TGV'].iloc[0]
        feature['properties']['TGV'] = tgv_value
    else:
        feature['properties']['TGV'] = 'Valeur inconnue'

with open('modified_communes.geojson', 'w') as file:
    json.dump(geojson_data, file)

In [417]:
df = data[data["Annee"] == 2021]
df = df[["Code INSEE" ,"Commune", "Prixm2Moyen", "SurfaceMoy", "Nb_mutations", "TGV"]].dropna(subset=["SurfaceMoy", "Nb_mutations"])
df.drop_duplicates(subset=["Code INSEE"], inplace=True)

paris = folium.Map(location = [46.8534, 2.3488], zoom_start=6, min_zoom=4, tiles = "https://{s}.tile.openstreetmap.fr/osmfr/{z}/{x}/{y}.png",
        attr = '&copy; OpenStreetMap France | &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors')

choropleth = folium.Choropleth(
geo_data= "modified_communes.geojson",
name="choropleth",
data= df,
columns=["Code INSEE", "Prixm2Moyen"],
key_on="feature.properties.code",
fill_color="YlOrRd",
fill_opacity=0.7,
line_opacity=0.5,
highlight = True,
legend_name="Engagement par département",
)
choropleth.geojson.add_to(paris)

result_indexed = df.set_index('Code INSEE')

for feature in choropleth.geojson.data['features']:
    Communes = feature['properties']['code']
    feature['properties']['Prixm2Moyen'] = '{:,.0f} €'.format(result_indexed.loc[Communes, 'Prixm2Moyen']).replace(',', ' ') if Communes in list(result_indexed.index) else 'Aucune données disponibles'
    feature['properties']['Nb_mutations'] = format(result_indexed.loc[Communes, 'Nb_mutations']) if Communes in list(result_indexed.index) else 'Aucune données disponibles'
    feature['properties']['SurfaceMoy'] = '{:,.0f} m2'.format(result_indexed.loc[Communes, 'SurfaceMoy']) if Communes in list(result_indexed.index) else 'Aucune données disponibles'
    

folium.GeoJsonTooltip(['nom', "Prixm2Moyen", 'SurfaceMoy', 'Nb_mutations', "TGV"]).add_to(choropleth.geojson)    

In [419]:
paris.save("map.html")

In [420]:
test = data[data["Annee"]==2014]

In [423]:
test.loc[test["Prixm2Moyen"].idxmax(),:]

Code INSEE                                        33459
Annee                                              2014
Nb_mutations                                          1
NbMaisons                                           1.0
NbApparts                                           0.0
PrixMoyen                                      921000.0
Prixm2Moyen                                     11512.0
SurfaceMoy                                         80.0
Commune                              SAINT-PEY-D'ARMENS
geo_point_2d    44.85488878990279, -0.11810715694623501
Nom_gare                                            NaN
voyageurs                                           NaN
code_ligne                                          NaN
TGV                                         Pas de gare
Name: 10551, dtype: object